In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
import os

# 기본 색상 설정
dark_bg = '#343541'
white = '#ffffff'
black = '#000000'

# 전체 배경 설정
style = {'description_width': '80px'}
widgets.Widget.layout.display = 'flex'

app_bg = widgets.VBox()
app_bg.layout = widgets.Layout(
    background_color=dark_bg,
    padding='20px'
)

# 데이터 저장소
todo_items = []

# 레이아웃
layout_input = widgets.Layout(width='100%', height='35px')
layout_button = widgets.Layout(width='100%', height='40px')
layout_half = widgets.Layout(width='49%', height='40px')
layout_progress = widgets.Layout(width='100%', height='20px')

# 위젯 정의
title_input = widgets.Text(placeholder='할 일 입력', layout=layout_input, style=style)
progress_input = widgets.IntSlider(min=0, max=100, value=0, layout=layout_input, style=style)
due_input = widgets.DatePicker(layout=layout_input, style=style)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], layout=layout_input, style=style)
add_button = widgets.Button(description='추가', layout=layout_button, button_style='success')
save_button = widgets.Button(description='저장', layout=layout_half, button_style='info')
load_button = widgets.Button(description='불러오기', layout=layout_half, button_style='warning')

# UI 배치 박스
todo_list_box = widgets.VBox([])
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=layout_progress)
overall_progress_label = widgets.Label(value='')

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = ""
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"최종 진행률\n{int(avg)}%"

def refresh_list():
    todo_list_box.children = []
    for idx, item in enumerate(todo_items):
        progress_bar = widgets.FloatProgress(
            value=item['progress'],
            min=0,
            max=100,
            layout=layout_progress,
            style={'bar_color': f"rgb({255-item['progress']*2.55:.0f}, {255-item['progress']*2.55:.0f}, {255-item['progress']*2.55:.0f})"}
        )
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, layout=layout_input)
        title = widgets.Label(value=f"# {idx+1}. {item['title']}", layout=layout_input)
        due = widgets.Label(value=f"마감일: {item['due'].strftime('%Y-%m-%d') if item['due'] else '-'}")
        priority = widgets.Label(value=f"중요도: {item['priority']}")
        complete_btn = widgets.Button(description='완료✅', layout=layout_half, button_style='success')
        delete_btn = widgets.Button(description='삭제❌', layout=layout_half, button_style='danger')
        progress_label = widgets.Label(value=f"{item['progress']}%")

        def update_progress(change, i=item, bar=progress_bar, label=progress_label):
            i['progress'] = change['new']
            bar.value = change['new']
            label.value = f"{change['new']}%"
            update_overall_progress()

        def mark_complete(btn, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def delete_task(btn, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        slider.observe(update_progress, names='value')
        complete_btn.on_click(mark_complete)
        delete_btn.on_click(delete_task)

        box = widgets.VBox([
            title, widgets.HBox([widgets.Label('진행률'), progress_bar]), slider,
            progress_label, due, priority, widgets.HBox([complete_btn, delete_btn])
        ], layout=widgets.Layout(padding='10px', background_color=white))
        todo_list_box.children += (box,)

def add_task(btn):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            data = json.load(f)
            todo_items.clear()
            for i in data:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i['priority']
                })
            update_overall_progress()
            refresh_list()
    except:
        print('불러오기 실패')

save_button.on_click(save_data)
load_button.on_click(load_data)

# 전체 UI 구성
ui = widgets.VBox([
    widgets.HTML('<h2 style="color:white">📁 DAVID mk30</h2>'),
    title_input,
    progress_input,
    due_input,
    priority_input,
    add_button,
    widgets.HBox([save_button, load_button]),
    widgets.Label('최종 진행률', layout=widgets.Layout(justify_content='center', color='white')),
    overall_progress_bar,
    overall_progress_label,
    todo_list_box
])

display(ui)
update_overall_progress()
refresh_list()
